In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
'''
ResBlock
'''
class ResBlk(nn.Module):

    def __init__(self, ch_in, ch_out, stride=1):
        # 通过stride减少参数维度
        super(ResBlk, self).__init__()

        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)


        # [b, ch_in, h, w] => [b, ch_out, h, w]
        self.extra = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=stride),
            nn.BatchNorm2d(ch_out)
        )

    def forward(self, x):
        '''
        :param x: [b, ch, h, w]
        :return:
        '''
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # short cut
        # extra module: [b, ch_in, h, w] => [b, ch_out, h, w]
        # element-wise add:
        out = self.extra(x) + out

        return out


'''
ResNet-18
'''
class ResNet18(nn.Module):

    def __init__(self, classes_num):
        super(ResNet18, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64)
        )

        # follow 4 blocks
        # [b, 64, h, w] => [b, 128, h/2, w/2]
        self.blk1 = ResBlk(64, 128, stride=2)
        # [b, 128, h/2, w/2] => [b, 256, h/4, w/4]
        self.blk2 = ResBlk(128, 256, stride=2)
        # [b, 256, h/4, w/4] => [b, 512, h/8, w/8]
        self.blk3 = ResBlk(256, 512, stride=2)
        # [b, 512, h/8, w/8] => [b, 512, h/16, w/16]
        self.blk4 = ResBlk(512, 512, stride=2)

        self.out_layer = nn.Linear(512*1*1, classes_num)

    def forward(self, x):

        # [b, 3, h, w] => [b, 64, h, w]
        x = F.relu(self.conv1(x))

        # [b, 64, h, w] => [b, 512, h/16, w/16]
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)

        # [b, 512, h/16, w/16] => [b, 512, 1, 1]
        x = F.adaptive_avg_pool2d(x, [1, 1])

        # [b, 512, 1, 1] => [b, 512]
        x = x.view(x.size(0), -1)
        # [b, 512] => [b, 10]
        x = self.out_layer(x)

        return x

In [3]:
loss = []
loss_test = []
acc = []
x_label = []

In [4]:
from torch import optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#  用CIFAR-10 数据集进行实验

def main():
    batchsz = 128

    cifar_train = datasets.CIFAR10('D:/研究生/暑期培训/NO1/data', True, transform=transforms.Compose([
    transforms.ToTensor(),  # 转化为tensor类型
    # 从[0,1]归一化到[-1,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(),  # 随机水平镜像
    transforms.RandomErasing(scale=(0.04, 0.2), ratio=(0.5, 2)),  # 随机遮挡
    transforms.RandomCrop(32, padding=4),  # 随机裁剪
    ]), download=False)
    cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

    cifar_test = datasets.CIFAR10('D:/研究生/暑期培训/NO1/data', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]), download=False)
    cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)

    x, label = iter(cifar_train).next()
    #print('x:', x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    #print(device)
    # model = Lenet5().to(device)
    model = ResNet18(10).to(device)
    #state_dict=torch.load('net_params.pth')
    #model.load_state_dict(state_dict)

    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    print(model)
    
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    N_CLASSES = 10
    class_correct = list(0. for i in range(N_CLASSES))
    class_total = list(0. for i in range(N_CLASSES))
    

    for epoch in range(30):
        
        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # [b, 3, 32, 32]
            # [b]
            x, label = x.to(device), label.to(device)

            logits = model(x)
            # logits: [b, 10]
            # label:  [b]
            # loss: tensor scalar
            loss_epoch = criteon(logits, label)

            # backprop
            optimizer.zero_grad()
            loss_epoch.backward()
            optimizer.step()
            
        loss.append(loss_epoch.item())

        print(epoch, 'loss:', loss_epoch.item())

        model.eval()
        with torch.no_grad():
            # test
            total_correct = 0
            total_num = 0
            for x, label in cifar_test:
                # [b, 3, 32, 32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b, 10]
                logits = model(x)
                loss_test_epoch = criteon(logits, label)

                #print(logits)
                # [b]
                pred = logits.argmax(dim=1)
                # [b] vs [b] => scalar tensor
                correct = torch.eq(pred, label).float().sum().item()
                total_correct += correct
                total_num += x.size(0)
                
                c = (pred == label).squeeze()

        
                for i in range(len(label)):
                    _label = label[i]
                    class_correct[_label] += c[i].item()
                    class_total[_label] += 1
            
            loss_test.append(loss_test_epoch.item())
            print(epoch, 'loss_test:', loss_test_epoch.item())
            acc_epoch = total_correct / total_num
            acc.append(acc_epoch)
            print(epoch, 'test acc:', acc_epoch)

            for i in range(N_CLASSES):
                print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
            
        x_label.append(len(acc))
    
    #print('loss', loss, 'acc', acc)
    torch.save(model.state_dict(),'net_params_withRes.pth')
    
    
    
    return model, x_label, loss, loss_test, acc

def show_loss():
    plt.plot(x_label, loss)
    plt.savefig(fname="loss_withRes.png")
    plt.close()
    
def show_lossTest():
    plt.plot(x_label, loss_test)
    plt.savefig(fname="lossTest_withRes.png")
    plt.close()

def show_acc():
    plt.plot(x_label, acc)
    plt.savefig(fname="acc_withRes.png")
    plt.close()

if __name__ == '__main__':
    model, x_label, loss, loss_test, acc = main()

ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk1): ResBlk(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (blk2): ResBlk(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=

18 loss: 0.6116542220115662
18 loss_test: 0.5717411041259766
18 test acc: 0.8378
Accuracy of plane : 77 %
Accuracy of   car : 88 %
Accuracy of  bird : 70 %
Accuracy of   cat : 60 %
Accuracy of  deer : 73 %
Accuracy of   dog : 63 %
Accuracy of  frog : 83 %
Accuracy of horse : 79 %
Accuracy of  ship : 86 %
Accuracy of truck : 81 %
19 loss: 0.6717935800552368
19 loss_test: 0.22733265161514282
19 test acc: 0.8571
Accuracy of plane : 78 %
Accuracy of   car : 88 %
Accuracy of  bird : 71 %
Accuracy of   cat : 60 %
Accuracy of  deer : 74 %
Accuracy of   dog : 63 %
Accuracy of  frog : 84 %
Accuracy of horse : 79 %
Accuracy of  ship : 86 %
Accuracy of truck : 81 %
20 loss: 0.3613773584365845
20 loss_test: 0.5947285890579224
20 test acc: 0.8577
Accuracy of plane : 78 %
Accuracy of   car : 88 %
Accuracy of  bird : 71 %
Accuracy of   cat : 61 %
Accuracy of  deer : 75 %
Accuracy of   dog : 63 %
Accuracy of  frog : 84 %
Accuracy of horse : 80 %
Accuracy of  ship : 87 %
Accuracy of truck : 82 %
21 los

In [5]:
import matplotlib.pyplot as plt

show_loss()
show_lossTest()
show_acc()

In [16]:
'''
ResBlock
'''
class ResBlk_withoutRes(nn.Module):

    def __init__(self, ch_in, ch_out, stride=1):
        # 通过stride减少参数维度
        super(ResBlk_withoutRes, self).__init__()

        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)

    def forward(self, x):
        '''
        :param x: [b, ch, h, w]
        :return:
        '''
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # short cut
        
        return out


'''
ResNet-18-withoutRes
'''
class ResNet18_withoutRes(nn.Module):

    def __init__(self, classes_num):
        super(ResNet18_withoutRes, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64)
        )

        # follow 4 blocks
        # [b, 64, h, w] => [b, 128, h/2, w/2]
        self.blk1 = ResBlk_withoutRes(64, 128, stride=2)
        # [b, 128, h/2, w/2] => [b, 256, h/4, w/4]
        self.blk2 = ResBlk_withoutRes(128, 256, stride=2)
        # [b, 256, h/4, w/4] => [b, 512, h/8, w/8]
        self.blk3 = ResBlk_withoutRes(256, 512, stride=2)
        # [b, 512, h/8, w/8] => [b, 512, h/16, w/16]
        self.blk4 = ResBlk_withoutRes(512, 512, stride=2)

        self.out_layer = nn.Linear(512*1*1, classes_num)

    def forward(self, x):

        # [b, 3, h, w] => [b, 64, h, w]
        x = F.relu(self.conv1(x))

        # [b, 64, h, w] => [b, 512, h/16, w/16]
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)

        # [b, 512, h/16, w/16] => [b, 512, 1, 1]
        x = F.adaptive_avg_pool2d(x, [1, 1])

        # [b, 512, 1, 1] => [b, 512]
        x = x.view(x.size(0), -1)
        # [b, 512] => [b, 10]
        x = self.out_layer(x)

        return x

In [32]:
loss_withoutRes = []
loss_test_withoutRes = []
acc_withoutRes = []
x_label_withoutRes = []

In [33]:
from torch import optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#  用CIFAR-10 数据集进行实验

def main():
    batchsz = 128

    cifar_train = datasets.CIFAR10('D:/研究生/暑期培训/NO1/data', True, transform=transforms.Compose([
    transforms.ToTensor(),  # 转化为tensor类型
    # 从[0,1]归一化到[-1,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(),  # 随机水平镜像
    transforms.RandomErasing(scale=(0.04, 0.2), ratio=(0.5, 2)),  # 随机遮挡
    transforms.RandomCrop(32, padding=4),  # 随机裁剪
    ]), download=False)
    cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

    cifar_test = datasets.CIFAR10('D:/研究生/暑期培训/NO1/data', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]), download=False)
    cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)

    x, label = iter(cifar_train).next()
    #print('x:', x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    #print(device)
    # model = Lenet5().to(device)
    model = ResNet18_withoutRes(10).to(device)
    #state_dict=torch.load('net_params.pth')
    #model.load_state_dict(state_dict)

    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    print(model)
    
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    N_CLASSES = 10
    class_correct = list(0. for i in range(N_CLASSES))
    class_total = list(0. for i in range(N_CLASSES))
    

    for epoch in range(30):
        
        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # [b, 3, 32, 32]
            # [b]
            x, label = x.to(device), label.to(device)

            logits = model(x)
            # logits: [b, 10]
            # label:  [b]
            # loss: tensor scalar
            loss_withoutRes_epoch = criteon(logits, label)

            # backprop
            optimizer.zero_grad()
            loss_withoutRes_epoch.backward()
            optimizer.step()
            
        loss_withoutRes.append(loss_withoutRes_epoch.item())

        print(epoch, 'loss:', loss_withoutRes_epoch.item())

        model.eval()
        with torch.no_grad():
            # test
            total_correct = 0
            total_num = 0
            for x, label in cifar_test:
                # [b, 3, 32, 32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b, 10]
                logits = model(x)
                loss_test_withoutRes_epoch = criteon(logits, label)

                #print(logits)
                # [b]
                pred = logits.argmax(dim=1)
                # [b] vs [b] => scalar tensor
                correct = torch.eq(pred, label).float().sum().item()
                total_correct += correct
                total_num += x.size(0)
                
                c = (pred == label).squeeze()
        
                for i in range(len(label)):
                    _label = label[i]
                    class_correct[_label] += c[i].item()
                    class_total[_label] += 1
            
            loss_test_withoutRes.append(loss_test_withoutRes_epoch.item())
            print(epoch, 'loss_test:', loss_test_withoutRes_epoch.item())
            acc_withoutRes_epoch = total_correct / total_num
            acc_withoutRes.append(acc_withoutRes_epoch)
            print(epoch, 'test acc:', acc_withoutRes_epoch)

            for i in range(N_CLASSES):
                print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
            
        x_label_withoutRes.append(len(acc_withoutRes))
    
    #print('loss', loss, 'acc', acc)
    torch.save(model.state_dict(),'net_params_withoutRes.pth')
    
    
    
    return model, x_label_withoutRes, loss_withoutRes, loss_test_withoutRes, acc_withoutRes

def show_loss():
    plt.plot(x_label_witoutRes, loss_witoutRes)
    plt.savefig(fname="loss_withoutRes.png")
    plt.close()
    
def show_lossTest():
    plt.plot(x_label_witoutRes, loss_test_witoutRes)
    plt.savefig(fname="lossTest_withoutRes.png")
    plt.close()

def show_acc():
    plt.plot(x_label_witoutRes, acc_witoutRes)
    plt.savefig(fname="acc_withoutRes.png")
    plt.close()

if __name__ == '__main__':
    model, x_label_withoutRes, loss_withoutRes, loss_test_withoutRes, acc_withoutRes = main()

ResNet18_withoutRes(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk1): ResBlk_withoutRes(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk2): ResBlk_withoutRes(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk3): ResBlk_withoutRes(
   

20 loss: 0.3911803364753723
20 loss_test: 0.1723327934741974
20 test acc: 0.8531
Accuracy of plane : 80 %
Accuracy of   car : 89 %
Accuracy of  bird : 71 %
Accuracy of   cat : 56 %
Accuracy of  deer : 72 %
Accuracy of   dog : 67 %
Accuracy of  frog : 84 %
Accuracy of horse : 80 %
Accuracy of  ship : 86 %
Accuracy of truck : 84 %
21 loss: 0.42608070373535156
21 loss_test: 0.22855743765830994
21 test acc: 0.8575
Accuracy of plane : 80 %
Accuracy of   car : 89 %
Accuracy of  bird : 72 %
Accuracy of   cat : 57 %
Accuracy of  deer : 72 %
Accuracy of   dog : 68 %
Accuracy of  frog : 85 %
Accuracy of horse : 80 %
Accuracy of  ship : 86 %
Accuracy of truck : 84 %
22 loss: 0.41460078954696655
22 loss_test: 0.6336594223976135
22 test acc: 0.8596
Accuracy of plane : 80 %
Accuracy of   car : 89 %
Accuracy of  bird : 72 %
Accuracy of   cat : 58 %
Accuracy of  deer : 73 %
Accuracy of   dog : 68 %
Accuracy of  frog : 85 %
Accuracy of horse : 80 %
Accuracy of  ship : 87 %
Accuracy of truck : 84 %
23 l

In [34]:
def show_loss_withoutRes():
    plt.plot(x_label_withoutRes, loss_withoutRes)
    plt.savefig(fname="loss_withoutRes.png")
    plt.close()
    
def show_lossTest_withoutRes():
    plt.plot(x_label_withoutRes, loss_test_withoutRes)
    plt.savefig(fname="lossTest_withoutRes.png")
    plt.close()

def show_acc_withoutRes():
    plt.plot(x_label_withoutRes, acc_withoutRes)
    plt.savefig(fname="acc_withoutRes.png")
    plt.close()
    
show_loss_withoutRes()
show_lossTest_withoutRes()
show_acc_withoutRes()

In [27]:
print(x_label_withoutRes)

[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
